В этой тетрадке я предпринимаю отчаянную попытку перевести на ржя через модель mt5

In [1]:
#!g1.1
%pip install transformers sentencepiece
%pip install simpletransformers
%pip install pympi-ling
%pip install razdel

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 249 kB 3.3 MB/s            
     |████████████████████████████████| 10.1 MB 42.2 MB/s            
     |████████████████████████████████| 342 kB 75.3 MB/s            
     |████████████████████████████████| 43 kB 2.1 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.2 MB 65.2 MB/s            
     |████████████████████████████████| 6.6 MB 69.9 MB/s            
     |████████████████████████████████| 29.4 MB 57.1 MB/s            
     |████████████████████████████████| 128 kB 94.5 MB/s            
     |████████████████████████████████| 78 kB 5.6 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 8

In [2]:
#!g1.1
%pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.4 MB 3.3 MB/s            
     |████████████████████████████████| 750.6 MB 5.9 kB/s              
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.5.10 requires setuptools==59.5.0, but you have setuptools 51.0.0 which is incompatible.
torchvision 0.10.1+cu111 requires torch==1.9.1, but you have torch 1.11.0 which is incompatible.
torchaudio 0.9.1 requires torch==1.9.1, but you have torch 1.11.0 which is incompatible.
You should consider upgrading via the '/usr/loca

In [ ]:
#!unzip Разметки.zip

In [2]:
#!g1.1
import pandas as pd
import numpy as np
import pympi
from razdel import tokenize
from sklearn.model_selection import train_test_split
#from torchtext.legacy import data
import torchtext.vocab as vocab
import torch

In [3]:
#!g1.1
# check if cuda is on
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False

True

## Загрузка данных

Загружаем данные, потом делим их на трейн и тест и сохраняем в соответствующие два файла

In [4]:
#!g1.1
import re
import json
import pymystem3
import os, sys
sys.path.append('../')
from argparse import Namespace

from pyeaf.pyeaf import EAFReader
from pyeaf.text import VocabularyVectorizer, TextStemmer, RSLStemmer, GramBinarizer

In [105]:
"""
def save_data(train, test, format_save="csv"):
    # Save rsl_sentences and rus_sentences to csv or json
    if format_save == 'json':
        with open('train.json', 'w', encoding='utf-8') as file:
            train.to_json(file, orient='records', lines=True, force_ascii=False)
        with open('test.json', 'w', encoding='utf-8') as file:
            test.to_json(file, orient='records', lines=True, force_ascii=False)
    elif format_save == 'csv':
        with open('train.csv', 'w', encoding='utf-8') as file:
            train.to_csv(file, index=False)
        with open('test.csv', 'w', encoding='utf-8') as file:
            test.to_csv(file, index=False)
"""

In [116]:
#raw_data = {'input_text': sentences_rus, 'target_text': sentences_rsl}
#df = pd.DataFrame(raw_data, columns=['input_text', 'target_text'])
#train, test = train_test_split(df, test_size=0.2)

#save_data(train, test)

In [117]:
#df.head()

,input_text,target_text
0,Здравствуйте!,здравствуйте
1,Скоро 23 февраля - мужской праздник.,скоро 23 февраль праздник мужчина2 poss:3ps
2,8 марта - женский праздник.,8 м-а-р-т праздник poss:3ps женщина
3,Мы отдыхаем.,мы отдых
4,В остальные дни работаем как обычно.,день другой:pl работать2 обычно


In [5]:
#!g1.1
generated_data = pd.read_csv('generated_stemed_less_repetitions.csv', encoding='utf-8')

generated_data['stem_rus'] = generated_data['new_stem_sent_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['rsl'] = generated_data['new_stem_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['gram_rus'] = generated_data['new_gram_sent_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [6]:
#!g1.1
for index, row in generated_data.iterrows():
    if row['rsl'] == ['']:
        generated_data.drop(index, inplace=True)

In [7]:
#!g1.1
test_data = pd.read_csv('test_data.csv', encoding='utf-8')

test_data['stem_rus'] = test_data['test_stem_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['rsl'] = test_data['test_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['gram_rus'] = test_data['test_gram_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [8]:
#!g1.1
for index, row in test_data.iterrows():
    if row['rsl'] == ['']:
        test_data.drop(index, inplace=True)

In [9]:
#!g1.1
generated_data.head()

,new_stem_sent_rus,new_gram_sent_rus,new_stem_rsl,stem_rus,rsl,gram_rus
3,"['рано', 'я', 'не', 'любить', 'молоко']","[['срав', 'ADV'], ['1-л', 'ед', 'им', 'SPRO'],...","['раньше', 'молоко', '1ps', 'не.любить']","[рано, я, не, любить, молоко]","[раньше, молоко, 1ps, не.любить]","[[срав, ADV], [1-л, ед, им, SPRO], [PART], [пе..."
4,"['рано', 'я', 'не', 'гражданин', 'животное']","[['срав', 'ADV'], ['1-л', 'ед', 'им', 'SPRO'],...","['раньше', 'животное', '1ps', 'гражданин']","[рано, я, не, гражданин, животное]","[раньше, животное, 1ps, гражданин]","[[срав, ADV], [1-л, ед, им, SPRO], [PART], [пе..."
5,"['рано', 'я', 'не', 'житель', 'наш', 'родной',...","[['срав', 'ADV'], ['1-л', 'ед', 'им', 'SPRO'],...","['раньше', 'животное', '1ps', 'житель', 'наш',...","[рано, я, не, житель, наш, родной, город, живо...","[раньше, животное, 1ps, житель, наш, родной, г...","[[срав, ADV], [1-л, ед, им, SPRO], [PART], [од..."
7,"['приезжать', 'в', 'деревня', 'а', 'там', 'нет...","[['изъяв', 'несов', 'непрош', 'нп', '1-л', 'V'...","['деревня', 'путешествие', ';', 'ничего3', 'эл...","[приезжать, в, деревня, а, там, нет, электриче...","[деревня, путешествие, ;, ничего3, электричест...","[[изъяв, несов, непрош, нп, 1-л, V, мн], [PR],..."
8,"['отвечать', 'в', 'рыба', 'много', 'жир', 'в',...","[['V', 'инф', 'несов', 'нп'], ['PR'], ['од', '...","['отвечать', 'путешествие', 'рыба', 'много', '...","[отвечать, в, рыба, много, жир, в, хвост, а, т...","[отвечать, путешествие, рыба, много, жир, в, х...","[[V, инф, несов, нп], [PR], [од, им, ед, род, ..."


Теперь мы можем настроить логгирование и загрузить модельку для русского вот отсюда:

https://huggingface.co/cointegrated/rut5-small

Я беру именно эту, потому что там только русский язык и моделька маленькая, значит, моя память это потянет.

In [19]:
#!g1.1
import logging
#from simpletransformers.t5 import T5Model, T5Args
from transformers import T5ForConditionalGeneration, T5Tokenizer

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-small")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small")

Downloading:   0%|          | 0.00/625k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/247M [00:00<?, ?B/s]

Теперь достаем данные снова

In [10]:
#!g1.1
generated_data['input_text'] = generated_data['stem_rus'].apply(' '.join)
generated_data['target_text'] = generated_data['rsl'].apply(' '.join)

In [11]:
#!g1.1
test_data['input_text'] = test_data['stem_rus'].apply(' '.join)
test_data['target_text'] = test_data['rsl'].apply(' '.join)

In [12]:
#!g1.1
generated_data['prefix'] = ''
test_data['prefix'] = ''

In [13]:
#!g1.1
test_data.head()

,test_stem_rus,test_gram_rus,test_rsl,stem_rus,rsl,gram_rus,input_text,target_text,prefix
0,"['а', 'когда', 'дед', 'мороз', 'вернуться', 'д...","[['CONJ'], ['CONJ'], ['им', 'мн', 'S', 'муж', ...","['indx', 'дед.мороз', 'clf', 'clf.группа', 'cl...","[а, когда, дед, мороз, вернуться, домой, мален...","[indx, дед.мороз, clf, clf.группа, clf.перемес...","[[CONJ], [CONJ], [им, мн, S, муж, од], [им, мн...",а когда дед мороз вернуться домой маленький де...,indx дед.мороз clf clf.группа clf.переместить ...,
1,"['а', 'мачеха', 'лишать', 'я', 'весь', 'радость']","[['CONJ'], ['им', 'S', 'од', 'жен', 'ед'], ['и...","['мама', 'второй', '3ps', 'все', 'радость', 'д...","[а, мачеха, лишать, я, весь, радость]","[мама, второй, 3ps, все, радость, давать, 1ps,...","[[CONJ], [им, S, од, жен, ед], [изъяв, V, пе, ...",а мачеха лишать я весь радость,мама второй 3ps все радость давать 1ps ноль,
4,"['зачем', 'я', 'хотеть', 'изменять', 'свой', '...","[['ADVPRO'], ['им', 'ед', '1-л', 'SPRO'], ['из...","['зачем', 'желание', 'изменение', 'все4', 'при...","[зачем, я, хотеть, изменять, свой, привычка, т...","[зачем, желание, изменение, все4, привычка, тр...","[[ADVPRO], [им, ед, 1-л, SPRO], [изъяв, V, пе,...",зачем я хотеть изменять свой привычка тратить ...,зачем желание изменение все4 привычка тратить ...,
5,"['оказываться', 'впервые', 'это', 'появляться'...","[['изъяв', 'V', '3-л', 'нп', 'несов', 'ед', 'н...","['америка', 'есть', 'poss', '3ps', '3ps', 'рас...","[оказываться, впервые, это, появляться, в, аме...","[америка, есть, poss, 3ps, 3ps, рассказ, 1ps, ...","[[изъяв, V, 3-л, нп, несов, ед, непрош], [ADV]...",оказываться впервые это появляться в америка,америка есть poss 3ps 3ps рассказ 1ps интересн...,
6,"['я', 'вчера', 'не', 'стирать', 'белье']","[['им', 'ед', '1-л', 'SPRO'], ['ADV'], ['PART'...","['1ps', 'вчера', 'стирать', '<dact>', 'не', 'б...","[я, вчера, не, стирать, белье]","[1ps, вчера, стирать, <dact>, не, был]","[[им, ед, 1-л, SPRO], [ADV], [PART], [изъяв, V...",я вчера не стирать белье,1ps вчера стирать <dact> не был,


Переходим к обучению

Зафайнтьюненная модель будет сохранена в папке `outputs` по дефолту. (https://simpletransformers.ai/docs/tips-and-tricks/#saving-checkpoints)

In [27]:
#!g1.1
import wandb

In [29]:
#!g1.1
wandb.login(relogin)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

In [14]:
#!g1.1
from simpletransformers.t5 import T5Model, T5Args

use_cuda = False

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 30
model_args.eval_batch_size = 30
model_args.num_train_epochs = 15
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Russian-RSL Translation"

model2 = T5Model("mt5", "cointegrated/rut5-small", args=model_args)

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/247M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

In [ ]:
# encodings_train = tokenizer.prepare_seq2seq_batch(list(train_df['input_text']), list(train_df['target_text']))

In [ ]:
# encodings_train.keys()

In [ ]:
# print(len(encodings_train['input_ids']), len(train_df))

In [15]:
#!g1.1
# Train the model
model2.train_model(generated_data, eval_data=test_data)

  0%|          | 0/8583 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: annaklezovich2 (use `wandb login --relogin` to force relogin)


Running Epoch 0 of 15:   0%|          | 0/287 [00:00<?, ?it/s]

...

Running Epoch 13 of 15:   0%|          | 0/287 [00:00<?, ?it/s]

  0%|          | 0/758 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/287 [00:00<?, ?it/s]

  0%|          | 0/758 [00:00<?, ?it/s]

(4305,
 {'global_step': [287,
   574,
   861,
   1148,
   1435,
   1722,
   2009,
   2296,
   2583,
   2870,
   3157,
   3444,
   3731,
   4018,
   4305],
  'eval_loss': [2.0065694818129907,
   1.75325935620528,
   1.5093205617024348,
   1.3799295379565313,
   1.2434252844407008,
   1.1754297201450055,
   1.0559051036834717,
   0.9567338113601391,
   0.9025575770781591,
   0.8501016314213092,
   0.7744979525987918,
   0.7454324146875968,
   0.7179433371012027,
   0.6817953884601593,
   0.6988227940522708],
  'train_loss': [2.3618712425231934,
   1.3919720649719238,
   2.0249407291412354,
   1.3653364181518555,
   1.2446787357330322,
   1.4440267086029053,
   0.5815677046775818,
   0.8137558102607727,
   0.9080497622489929,
   0.7557668089866638,
   0.6037885546684265,
   0.4817356765270233,
   0.4576496183872223,
   0.5287657976150513,
   0.7542520761489868]})

Вот, что-то обучили...

Попробовала на 5, 15 и 10 эпох. На 5 эпох получились самые лучшие результаты.

P.S. Забыла про дурацкую токенизацию ржя, пришлось еще раз переобучить, в этот раз сделала на 7 эпох.


Для модели на аугментированных данных 30 эпох слишком много, судя по картинке на wandb после 22 эпохи начинается переобучение.

Теперь можно оценить и протестировать (пока тестирую модель на 10)

In [36]:
#!g1.1
%pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [37]:
#!g1.1
#import logging
#import sacrebleu
#from simpletransformers.t5 import T5Model, T5Args


#logging.basicConfig(level=logging.INFO)
#transformers_logger = logging.getLogger("transformers")
#transformers_logger.setLevel(logging.WARNING)

#model_args = T5Args()
#model_args.max_length = 512
#model_args.length_penalty = 1
#model_args.num_beams = 10

#model2 = T5Model("mt5", "outputs", args=model_args)

Failed to deserialize variable 'model_args'. Run the following code to delete it:
  del_datasphere_variables('model', 'model2', 'model_args')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py", line 282, in __setstate__
    self.sp_model.Load(self.vocab_file)
  File "/home/jupyter/.local/lib/python3.8/site-packages/sentencepiece.py", line 367, in Load
    return self.LoadFromFile(model_file)
  File "/home/jupyter/.local/lib/python3.8/site-packages/sentencepiece.py", line 177, in LoadFromFile
    return _sentencepiece.SentencePieceProcessor_LoadFromFile(self, arg)
OSError: Not found: "/tmp/xdg_cache/huggingface/transformers/a70e10655dd799aac70d77cb3c0e2ed2dbc7e0e9e5a5fdcea95b424e5c7696fc.49f19f2d88f5168206d686d69bd0f6495942381027a2decd2658470f4e669fb9": No such file or

Попробуем оценить, что произошло

In [16]:
#!g1.1
model2.eval_model(eval_data=test_data)

  0%|          | 0/758 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/26 [00:00<?, ?it/s]

{'eval_loss': 0.6988227940522708}

In [17]:
#!g1.1
truth_rsl = test_data["target_text"].tolist()
to_rsl = test_data["input_text"].tolist()

In [18]:
#!g1.1
import sacrebleu

rsl_preds = model2.predict(to_rsl)
rus_rsl_bleu = sacrebleu.corpus_bleu(rsl_preds, [truth_rsl])
print("--------------------------")
print("Russian to RSL: ", rus_rsl_bleu.score)

Generating outputs:   0%|          | 0/26 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/758 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.8/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9ed2c9a160> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  warnings.warn("backward hook {} on tensor will not be "
/home/jupyter/.local/lib/python3.8/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9ed2c9a280> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  warnings.warn("backward hook {} on tensor will not be "
/home/jupyter/.local/lib/python3.8/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9ed2c9a3a0> o

--------------------------
Russian to RSL:  27.061430610729374


In [35]:
#!g1.1
rsl_preds[:10]

['indx дед.мороз clf clf.группа clf.перемест',
 'мама второй 3ps все радость давать 1ps ноль',
 'зачем желание изменение все4 привычка тратить все4 желание',
 'америка есть poss 3ps 3ps рассказ 1ps интересный',
 '1ps вчера стирать <dact> не был',
 'обувь2 убирать чулок бродить дверь 1ps думать радость',
 '<dact> дальше <dact>',
 'начинать солнце друг уже рядом река',
 '1ps все 1ps устраивать 1ps интересный работать2 долго 1',
 'мы отдых']

In [36]:
#!g1.1
truth_rsl[:10]

['indx дед.мороз clf clf.группа clf.переместить домой clf clf.группа clf.переместить дед.мороз маленький встретить приходить обращение главный 1ps был лес подарок clf clf.перечислять',
 'мама второй 3ps все радость давать 1ps ноль',
 'зачем желание изменение все4 привычка тратить все4 желание именение привычка свой изменение тратить и копить думать',
 'америка есть poss 3ps 3ps рассказ 1ps интересный америка первый прогресс',
 '1ps вчера стирать <dact> не был',
 'обувь2 убирать чулок бродить дверь 1ps думать радость 3ps скоро открывать.дверь сейчас скоро радость сердцебиение',
 '<dact> дальше <dact>',
 'утро солнце друг уже рядом река',
 '1ps спокойный2 1ps был интересный учиться и время clf clf.время clf.идет 1ps уже сам2 получать даже сложный 3ps заказывать 1ps',
 'мы отдых']

**Результат:**

Russian to RSL (на неправильльной токенизации ржя):  28.527910962357836

Но как бы сравнить не с чем. Хотя чиселка вроде немаленькая такая

Но при этом почему-то, когда я переехала на более корректную предобработку русского, качество модели резко упало

Russian to RSL:  5.3837717877066265

Та же тетрадка, но replacements augmentation strategy не больше 4 повторений одной структуры:

* Russian to RSL:  18.513365924823393  на 10 эпохах
* Russian to RSL:  42.435910660455576 на 30 эпохах, судя по графику после 22 эпохи происходит overfit
* Russian to RSL:  27.061430610729374 на 15 эпохах, because we want to take something on the edge of the eval_fit curve, right before it flatlines

In [39]:
#!g1.1
for x, y in zip(rsl_preds[:2], truth_rsl[:2]):
    print(x + '\t' + '===' + '\t' + y)

indx дед.мороз clf clf.группа clf.перемест	===	indx дед.мороз clf clf.группа clf.переместить домой clf clf.группа clf.переместить дед.мороз маленький встретить приходить обращение главный 1ps был лес подарок clf clf.перечислять
мама второй 3ps все радость давать 1ps ноль	===	мама второй 3ps все радость давать 1ps ноль


In [38]:
#!g1.1
rsl_preds[0], truth_rsl[0]

('indx дед.мороз clf clf.группа clf.перемест',
 'indx дед.мороз clf clf.группа clf.переместить домой clf clf.группа clf.переместить дед.мороз маленький встретить приходить обращение главный 1ps был лес подарок clf clf.перечислять')

In [40]:
#!g1.1
with open("predict_20.tsv", "w", encoding="utf-8") as f:
    f.write('Russian' + '\t' + 'Predicted' + '\t' + 'Truth' + '\n')
    for rus, preds, real in zip(to_rsl, rsl_preds, truth_rsl):
        f.write(rus + '\t' + preds + '\t' + real + '\n')